# **Sentiment Analysis of Product Reviews - Machine Learning**

### Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

### Inputs

* Write down which data or information you need to run the notebook 

### Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

### Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

## Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\admin\\Documents\\vscode-projects\\ci-hackathon-sentiment-analysis-of-product-reviews\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\admin\\Documents\\vscode-projects\\ci-hackathon-sentiment-analysis-of-product-reviews'

---

## Initial Setup

Import relevant libraries and extract cleaned sample dataset

In [5]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re  

In [6]:
df = pd.read_csv('data/Reviews_Sample_Cleaned.csv')
print("Data loaded successfully")
print(df.shape)
df.head()

Data loaded successfully
(20000, 12)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Date,Sentiment
0,406806,B000LKXCB8,ALFO2XQISZ9QM,A.K.,1,1,1,1342396800,.,"What a waste!!! Hardly any ""Mirch"" (Green Chil...",2012-07-16,Negative
1,75918,B001CPOR2E,A7G9M0IE7LABX,Kevin,2,3,1,1310947200,Treat caused ulcers in my dog's GI tract,I should have known when I touched these strip...,2011-07-18,Negative
2,400895,B0016PC4TS,A22M0C1YXBGW1Y,"LuLu, Calif",8,10,1,1274486400,Waste of Money,Don't waste your money! Did nothing to repell ...,2010-05-22,Negative
3,480705,B00719WZNS,A2PA1INHPVXLBF,THE DUDE,0,0,1,1350086400,"ALL FIZZ, NOT MUCH FLAVOR",I RECIEVED MY DRINKS AND WAS LOOKING FOR LOTS ...,2012-10-13,Negative
4,103600,B005MSH01C,A3CZ8V0M91JEKP,unsatisfied user,4,5,1,1307404800,Grossest cola ever,"I've had the SodaStream SodaMix - Cola before,...",2011-06-07,Negative


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
